In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
# from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
import os
import json
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait

# Sessão de variaveis
path_to_downloads = "C:\\Users\\MrRobot\\Downloads\\"
path_to_musics = "C:\\Users\\MrRobot\\Music\\"
service = Service(executable_path="./chromedriver.exe")
options = webdriver.ChromeOptions()
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36' # se tiver "headless" no user_agent o site bloqueia o acesso
options.add_argument('user-agent={0}'.format(user_agent))
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('--mute-audio')
# options.add_argument("user-data-dir=C:\\Users\\MrRobot\\AppData\\Local\\Google\\Chrome\\User Data\\Profile 4") #Path to your chrome profile
c = webdriver.Chrome(service=service, options=options)
wdw = WebDriverWait(c, 20, 2)

def pegando_urls_no_ytb(link):
    """crawl da playlist do youtube e retorna ela em DICT"""
    try:
        c.get(link)

        sleep(20)
        videos = c.find_elements(By.XPATH, "//a[@id = 'video-title']")
    
        urls = []
        for hrefs in videos:
            dict = {}
            dict['title'] = hrefs.get_attribute("innerHTML").replace('\n          ', '').replace('\n        ', '')
            dict['url'] = hrefs.get_attribute("href")
            urls.append(dict)

        print(f'total de URLs: {len(urls)}')
        return urls
    except:
        return "ERRO ao pegar URLS da playlist no Youtube."

def automacao_only_mp3(url_msc):
        """Recebe uma url de um video e Faz as ações dentro do site de download"""
        site = "https://pt.onlymp3.to/38/"
        c.get(site)

        search_bar = c.find_element(By.ID, "txtUrl")
        search_bar.send_keys(url_msc)
        
        sleep(1)
        convert_btn = c.find_element(By.CLASS_NAME, "start-btn")
        convert_btn.click()

        wdw.until(lambda c: c.find_element(By.XPATH, "//button[@class='btn']"), "Esperando btn de download")

        print('espera acabou')
        sleep(1)
        download_btn = c.find_element(By.XPATH, "//button[@class='btn']")
        download_btn.click()

def eh_hora_de_reinciar(numero):
    """
        Função que reinicia o browser a cada 10 download

        obs: atrelar a mesma variável utilizada no inicio, nesse caso c = eh_hora_de_reinciar(10)
    """
    mbdir = os.listdir(path_to_downloads)
    mbdir.remove('desktop.ini')
    print('{} músicas sendo baixadas...'.format(numero))

    if numero == 1:
        # espera
        c.quit()
        print('browser reiniciando')
    else:
        print('não é hr de reiniciar o browser')

def espera_download():
    """ Pergunta: Tem algum item em processo de download ? """
    sleep(3)
    mb = os.listdir(path_to_downloads)
    mb.remove('desktop.ini')
    for item in mb:
        if ".crdownload" in item:
            print(item)
            return True
    return False

def downloads_suspeitos(playlist):
    """ Retorna itens q não foram confirmados na area de download"""
    confirmados = []
    lista_organizada = []
    mb = os.listdir(path_to_downloads)
    mb.remove('desktop.ini')

    for item in playlist:
        for msc in mb:
            if item['title'] in msc:
                confirmados.append(item) # items confirmados

    print('confimados: {}'.format(len(confirmados)))
    for item in confirmados:
        playlist.remove(item)

    for index, item in enumerate(playlist):
        lista_organizada.append(item['title'])
    return playlist, lista_organizada

def renomear_arquivos():
    """Renomeia os arquivos no diretório de downloads"""
    mb = os.listdir(path_to_downloads)
    mb.remove('desktop.ini')

    for i in mb:
        new_i = i.replace('onlymp3.to - ', '')
        os.replace(path_to_downloads+i, f'{path_to_downloads}{new_i[:-35]}.mp3')

def mover_arquivos():
    """mover os arquivos do diretório de downloads para music"""

    mb = os.listdir(path_to_downloads)
    mb.remove('desktop.ini')

    for i in mb:
        os.replace(path_to_downloads+i, f'{path_to_musics}{i}')



In [9]:
urls_ytb = [
  {'title': 'Tem cabaré essa noite - Nivaldo Marques | Nattan (Clipe Oficial)',
  'url': 'https://www.youtube.com/watch?v=WiNYFwIfg4w&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=1'},
 {'title': 'Ana Castela - Pipoco ft. @MELODY OFICIAL  e @DJ Chris no Beat (Clipe Oficial)',
  'url': 'https://www.youtube.com/watch?v=69JAoslGYI8&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=2'},
 {'title': 'Jesslee x @Justin Champagne x @Ryan Robinette - Chillbilly (Official Music Video)',
  'url': 'https://www.youtube.com/watch?v=ExC8xLA3cZM&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=3'},
 {'title': 'Bailey Zimmerman - Fall In Love (Official Music Video)',
  'url': 'https://www.youtube.com/watch?v=d1yKrnPHAmo&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=4'},
 {'title': 'Wesley Safadão - Amor ou Esquema - Garota Vip Manaus',
  'url': 'https://www.youtube.com/watch?v=XdyRZrVYTEc&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=5'},
 {'title': 'MC Paiva e MC Ryan SP - Casei Com a Putaria (Love Funk) Kotim',
  'url': 'https://www.youtube.com/watch?v=8NjXVsLHIg8&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=6'},
 {'title': 'FREAK ON - We Got The Fire',
  'url': 'https://www.youtube.com/watch?v=nlD3zta1bnE&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=7'},
 {'title': 'Gusttavo Lima - Fala Mal de Mim (Ao Vivo em Maceió)',
  'url': 'https://www.youtube.com/watch?v=hOGuaMazCz8&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=8'},
 {'title': 'Cole Swindell - She Had Me At Heads Carolina (Official Music Video)',
  'url': 'https://www.youtube.com/watch?v=pW98g-Fb-EU&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=9'},
 {'title': 'L7NNON &amp; BIEL DO FURDUNCINHO - Ai Preto 🤤 (part. Bianca)',
  'url': 'https://www.youtube.com/watch?v=NbB6qMA2lts&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=10'},
 {'title': 'Lamorn - Who We Are (Official Music Video)',
  'url': 'https://www.youtube.com/watch?v=XJOkM2RvjQ4&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=11'},
 {'title': 'Gusttavo Lima - Termina Comigo Antes  (Ao Vivo em Porto Alegre)',
  'url': 'https://www.youtube.com/watch?v=rTJSWmgbVwA&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=12'},
 {'title': 'FULLJOS - Monotonia',
  'url': 'https://www.youtube.com/watch?v=F1ahwvpRMTY&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=13'},
 {'title': 'Guilherme e Benuto feat @Hugo e Guilherme  - Haja Colírio | DVD Deu Rolo',
  'url': 'https://www.youtube.com/watch?v=qjxNM9qF3LA&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=14'},
 {'title': 'Mari Fernandez - COMUNICAÇÃO FALHOU feat. Nattan (DVD Ao Vivo em Fortaleza)',
  'url': 'https://www.youtube.com/watch?v=psdZRggL7cI&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=15'},
 {'title': 'Blake Shelton - No Body (Official Music Video)',
  'url': 'https://www.youtube.com/watch?v=JDhDy9rPpiE&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=16'},
 {'title': 'Kevi Jonny - Te Love (DVD Com Amor, Kevi Jonny)',
  'url': 'https://www.youtube.com/watch?v=vBQbH4mA_z0&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=17'},
 {'title': 'Colton James "I Miss America" Official Lyric Video',
  'url': 'https://www.youtube.com/watch?v=2vuhKROJQ3c&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=18'},
 {'title': 'Ingrid Andress - Feel Like This (Official Music Video)',
  'url': 'https://www.youtube.com/watch?v=0PeaTboS3_8&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=19'},
 {'title': 'Hugo e Guilherme, Marília Mendonça - Mal Feito - DVD Próximo Passo',
  'url': 'https://www.youtube.com/watch?v=kyfXEMqvLgU&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=20'},
 {'title': 'Ingrid Andress - Wishful Drinking (With Sam Hunt) (Official Music Video)',
  'url': 'https://www.youtube.com/watch?v=lE5WYXtOt7E&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=21'},
 {'title': 'Zak Meow   Acid Damage VISUALIZER',
  'url': 'https://www.youtube.com/watch?v=hTXBckKHVJ0&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=22'},
 {'title': 'SET Tatuapé Conceito 2 - Ryan SP, Kako, Joãozinho VT, Marks, GP e MC Erik (GR6 Explode) DJ Boy e GM',
  'url': 'https://www.youtube.com/watch?v=nbTAKkjnV_Q&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=23'},
 {'title': 'Gabby Barrett - Pick Me Up (Official Music Video)',
  'url': 'https://www.youtube.com/watch?v=0qG4chf4fLI&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=24'},
 {'title': 'Felipe Amorim - Gostosinha É Ela (Clipe Oficial)',
  'url': 'https://www.youtube.com/watch?v=zSRjeUpQUwI&list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW&index=25'}
]

len(urls_ytb)

25

Processo de execução

In [10]:
link = 'https://www.youtube.com/playlist?list=PL1icZgrADGk480H9rtFVC6C5D5DnoMChW'
urls_ytb = pegando_urls_no_ytb(link)
contador = 0
for msc in urls_ytb:
    contador += 1
    automacao_only_mp3(msc['url'])
    dez_msc = contador % 10

    if dez_msc == 0:
        while espera_download() == True:
            sleep(3)
        c.quit()
        c = webdriver.Chrome(service=service, options=options)
        wdw = WebDriverWait(c, 20, 2)
    
    print(contador)

while espera_download() == True:
    sleep(3)
c.quit()

total de URLs: 5
espera acabou
1
espera acabou
2
espera acabou
3
espera acabou
4
espera acabou
5
onlymp3.to - Guilherme e Benuto - 3 Batidas (DVD AMANDO BEBENDO E SOFRENDO)-RJNrSvFMHxs-192k-1654265385990.mp3.crdownload
onlymp3.to - Guilherme e Benuto - 3 Batidas (DVD AMANDO BEBENDO E SOFRENDO)-RJNrSvFMHxs-192k-1654265385990.mp3.crdownload


In [11]:

renomear_arquivos()
urls_ytb2 = urls_ytb.copy()
suspeitos = downloads_suspeitos(urls_ytb2)
sorted(suspeitos[1]) # Lista possíveis falhas

confimados: 3


['Guilherme e Benuto feat @Hugo e Guilherme  - Haja Colírio | DVD Deu Rolo',
 'Jorge &amp; Mateus - Então Valeu (Clipe Oficial) [É Simples Assim]']

In [12]:
suspeitos[0] # ache as urls q falharam e faça manualmente

[{'title': 'Guilherme e Benuto feat @Hugo e Guilherme  - Haja Colírio | DVD Deu Rolo',
  'url': 'https://www.youtube.com/watch?v=qjxNM9qF3LA&list=PLB5S3eu0mZof9705EtpG8SeFACN2itGRP&index=1'},
 {'title': 'Jorge &amp; Mateus - Então Valeu (Clipe Oficial) [É Simples Assim]',
  'url': 'https://www.youtube.com/watch?v=z-XxedqPsqg&list=PLB5S3eu0mZof9705EtpG8SeFACN2itGRP&index=4'}]

In [13]:
mover_arquivos() 

In [14]:
# Faça download manual dos arquivos q 'falharam' em:
"https://pt.onlymp3.to/38/"
# renomear_arquivos
# mover_arquivos() 

'https://pt.onlymp3.to/38/'